 <h1 style="background-color:black
;font-family:Arial white;font-size:150%;text-align:center;border-radius: 15px 20px;">   Interview - LLM Finetuning with LORA </h1><a id=0></a>
<div style="text-align:center;"> by </div>
<div style="text-align:center;">
YAHAYA Abdullahi</div>

<div style="text-align:center;">
    <img src="lora.png" height="250">
</div>


# Library Importation

In [ ]:
!pip install transformers -U
!pip install -q pypdf
!pip install -q python-dotenv
!pip install  llama-index
!pip install -q gradio
!pip install einops
!pip install accelerate
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-fastembed
!pip install fastembed
!pip install --upgrade tokenizers==0.19.*


In [2]:
%%capture
import logging
import sys
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import PromptTemplate

import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass
from transformers import AutoTokenizer
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import TextStreamer

import warnings
warnings.filterwarnings('ignore')

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Load Builing data from location

In [3]:
documents = SimpleDirectoryReader("Data").load_data()
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model
Settings.chunk_size = 512

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

## Query prompt config

In [4]:
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

## Authentication of Token

In [5]:
from huggingface_hub import  notebook_login
notebook_login()

## LLM Model initilization and training

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct"
)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

llm = HuggingFaceLLM(
    context_window=8192,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    stopping_ids=stopping_ids,
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

Settings.llm = llm
Settings.chunk_size = 512

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device

## Conversion of Document into Vector

In [ ]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

def predict(input, history):
  response = query_engine.query(input)
  return str(response)


## GUI Interface for interaction

In [ ]:
import gradio as gr

gr.ChatInterface(predict).launch(share=True)

## Sample queries to be compared

In [ ]:
sample_queries = [
    "What are the different types of insulation materials used in buildings?",
    "What are the benefits of using LED lighting in buildings?",
    "What factors contribute to indoor air quality in buildings?",
    "Explain the concept of building automation and its advantages."
]

for query in sample_queries:
    response = predict(query, history=None)  
    print("Query:", query)
    print("Response:", response)
    print()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Query: What are the different types of insulation materials used in buildings?
Response: Based on the provided context, it seems that the text does not explicitly mention the different types of insulation materials used in buildings. However, it does mention the concept of thermal resistance (R) and thermal conductivity (k) of materials, which are important properties of insulation materials. Additionally, it mentions the use of insulation in the context of envelope heat transfer (Q) and thermal mass. If you're looking for specific types of insulation materials, I'd be happy to help you with that. Please provide more context or clarify what you're looking for.



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Query: What are the benefits of using LED lighting in buildings?
Response: LED lighting can help reduce energy consumption by reducing the number of hours electric lighting systems are energized, which in turn reduces sensible lighting heat gains and mechanical cooling demands. Additionally, LED lighting can provide health and wellness benefits to building occupants by providing views to outdoors and exposure to natural daylight.



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Query: What factors contribute to indoor air quality in buildings?
Response:  According to the provided context, factors that contribute to indoor air quality in buildings include outdoor air supply rates, indoor CO2 concentration, and air ventilation rates. Additionally, the Clean First framework presented in the paper suggests a system-level approach to achieve sustainable indoor air quality, which includes layered air cleaning strategies. Furthermore, ASHRAE Standard 55 defines the combinations of measurable indoor thermal environmental factors and personal factors that produce thermal environmental conditions acceptable to a majority of occupants within a space. These factors include air speed, relative humidity, air temperature, radiant temperature, and clothing level.

Query: Explain the concept of building automation and its advantages.
Response: Building automation refers to the use of technology to control and monitor various building systems, such as heating, ventilation, and

### Lora Fine tuning


In [ ]:
from unsloth import FastLanguageModel
max_seq_length = 2048 
dtype = None
load_in_4bit = True 
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


## Finetune Dataset 

In [ ]:
build_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # do not forget this part!
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = build_prompt.format(instruction, input, output) + EOS_TOKEN # without this token generation goes on forever!
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("Allmen/Building_technology/building_technology_questions_with_responses.json", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

## Parameters Tuning

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # increase this to make the model learn "better"
        num_train_epochs=4,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

## Training of finetuned model

In [ ]:
trainer_stats = trainer.train()

## Testing the pretrained model

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    build_prompt.format(
        "What are some factors to consider when choosing windows materials?.", # instruction
        "walls, roofs, floors", # input
        "", # output 
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

## Saving the model output

In [ ]:
model.save_pretrained("lora_model") # Local saving
model.push_to_hub("yourhuggingID/lora_model", token = "insert your key")